In [42]:
import pandas as pd
import numpy as np
import sklearn as sk
import xgboost
import pickle
import lightgbm as lgb

from sklearn import linear_model
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn.metrics import explained_variance_score, mean_absolute_error
import matplotlib.pyplot as plt

%matplotlib inline

In [45]:
df_prop_2016 = pd.read_csv('/Users/fangjie/Downloads/properties_2016.csv')
df_prop_2017 = pd.read_csv('/Users/fangjie/Downloads/properties_2017.csv')
df_prop = df_prop_2017


In [ ]:
df_prop_2016.head()

In [41]:
df_trans_2016 = pd.read_csv('/Users/fangjie/Downloads/train_2016_v2.csv', parse_dates=[2])
df_trans_2017 = pd.read_csv('/Users/fangjie/Downloads/train_2017_v2.csv', parse_dates=[2])
df_trans = pd.concat((df_trans_2016, df_trans_2017), axis = 0)


IOError: File /Users/fangjie/Downloads/train_2017_v2.csv does not exist

In [ ]:
df_trans_2016.head()

In [ ]:
df_trans_2016.describe()

In [ ]:
df_prop_2016.describe()

In [ ]:
df_trans.transactiondate.hist()

In [15]:
df_prop_2016.dtypes

parcelid                          int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                   object
heatingorsystemtypeid           float64
latitude                        float64


In [14]:
df_prop_2016.columns

Index([u'parcelid', u'airconditioningtypeid', u'architecturalstyletypeid',
       u'basementsqft', u'bathroomcnt', u'bedroomcnt', u'buildingclasstypeid',
       u'buildingqualitytypeid', u'calculatedbathnbr', u'decktypeid',
       u'finishedfloor1squarefeet', u'calculatedfinishedsquarefeet',
       u'finishedsquarefeet12', u'finishedsquarefeet13',
       u'finishedsquarefeet15', u'finishedsquarefeet50',
       u'finishedsquarefeet6', u'fips', u'fireplacecnt', u'fullbathcnt',
       u'garagecarcnt', u'garagetotalsqft', u'hashottuborspa',
       u'heatingorsystemtypeid', u'latitude', u'longitude',
       u'lotsizesquarefeet', u'poolcnt', u'poolsizesum', u'pooltypeid10',
       u'pooltypeid2', u'pooltypeid7', u'propertycountylandusecode',
       u'propertylandusetypeid', u'propertyzoningdesc',
       u'rawcensustractandblock', u'regionidcity', u'regionidcounty',
       u'regionidneighborhood', u'regionidzip', u'roomcnt', u'storytypeid',
       u'threequarterbathnbr', u'typeconstructiontyp

# # check duplicates for each parcelID

In [18]:
df_prop_2016.parcelid.unique().shape[0] == df_prop_2016.parcelid.shape[0]


True

In [ ]:
# put just treat houses that are sold multiple times as different hosues for simplication purpose
(df_trans.parcelid.value_counts() > 1).sum() * 1.0 / df_trans.shape[0]

## merge data

In [19]:
df = pd.merge(left=df_prop_2016, right=df_trans_2016, on='parcelid')
df_pred = pd.merge(left=df_prop, right=df_trans.drop_duplicates(subset='parcelid',keep='first'), on='parcelid', how='left')

## pre-processing

In [23]:
df.censustractandblock.dtype

dtype('float64')

In [29]:
df.rawcensustractandblock.head().astype('string')

0    61110022.003
1    61110015.031
2    61110007.011
3    61110008.002
4    61110014.021
Name: rawcensustractandblock, dtype: object

# define features

In [ ]:
all_fields = set(df.columns)

# these are fields that are used to identify fields
identifiers = set(['transactiondate', 'parcelid'])

# log error that we want to model
label = set(['logerror'])

# the following are categorical features
feat_objects = set(df.columns[df.dtypes==object])
#  ['taxdelinquencyflag',
#  'propertycountylandusecode',
#  'propertyzoningdesc',
#  'fireplaceflag',
#  'hashottuborspa']

# the following are numerical features that should be treated as categorical features
feat_numeric_to_categorical = set([
    'airconditioningtypeid',
    'architecturalstyletypeid',
    'buildingqualitytypeid',
    'buildingclasstypeid',
    'decktypeid',
    'fips',
    'heatingorsystemtypeid',
    'propertylandusetypeid',
    'regionidcounty',
    'regionidcity',
    'regionidzip',
    'regionidneighborhood',
    'storytypeid',
    'typeconstructiontypeid',
    'yearbuilt',
    'assessmentyear',
    'taxdelinquencyyear'
])


# the rest are numeric features
feat_numeric = set([
    'basementsqft',
    'bathroomcnt',
    'bedroomcnt',
    'calculatedbathnbr',
    'threequarterbathnbr',
    'finishedfloor1squarefeet',
    'calculatedfinishedsquarefeet',
    'finishedsquarefeet6',
    'finishedsquarefeet12',
    'finishedsquarefeet13',
    'finishedsquarefeet15',
    'finishedsquarefeet50',
    'fireplacecnt',
    'fullbathcnt',
    'garagecarcnt',
    'garagetotalsqft',
    'hashottuborspa',
    'lotsizesquarefeet',
    'numberofstories',
    'poolcnt',
    'poolsizesum',
    'pooltypeid10',
    'pooltypeid2',
    'pooltypeid7',
    'roomcnt',
    'unitcnt',
    'yardbuildingsqft17',
    'yardbuildingsqft26',
    'taxvaluedollarcnt',
    'structuretaxvaluedollarcnt',
    'landtaxvaluedollarcnt',
    'taxamount'
])

# fields that are thrown away for now
feat_for_consideration_later = set([
    'latitude',
    'longitude',
    'rawcensustractandblock',
    'censustractandblock',
])

In [ ]:
eats_categorical = feat_numeric_to_categorical | feat_objects
feat_numeric = feat_numeric
feat = feat_categorical | feat_numeric

feat_categorical = list(feat_categorical)
feat_numeric = list(feat_numeric)
feat = list(feat)

# #convert categorical features

In [ ]:
# need to impute all these with a `missing` value first
df[feat_categorical] = df[feat_categorical].fillna('--unknown--')
df_pred[feat_categorical] = df_pred[feat_categorical].fillna('--unknown--')

In [ ]:
# use label encoder to encode all categorical variables
le = LabelEncoder()
for feat in feat_categorical:
    df_pred[[feat]] = df_pred[[feat]].apply(le.fit_transform)
    df[[feat]] = df[[feat]].apply(le.transform)

## impute missing values for numeric values

In [ ]:
imp =  Imputer(missing_values=np.nan,strategy="median",axis=0)

df_pred[feat_numeric] = imp.fit_transform(df_pred[feat_numeric])
df[feat_numeric] = imp.transform(df[feat_numeric])

## split train-test

In [ ]:
mask_train = df.transactiondate < '2017-06-01'
# mask_test_oct = (df.transactiondate >= '2016-10-01') & (df.transactiondate < '2016-11-01')
# mask_test_nov = (df.transactiondate >= '2016-11-01') & (df.transactiondate < '2016-12-01')
# mask_test_dec = (df.transactiondate >= '2016-12-01') & (df.transactiondate < '2017-01-01')
# mask_test = mask_test_oct | mask_test_nov | mask_test_dec
mask_test = (df.transactiondate >= '2017-06-01')

In [ ]:
X_train = df.loc[mask_train, feats].astype(float).values
X_test = df.loc[mask_test, feats].astype(float).values

y_train = np.array(df.loc[mask_train, 'logerror'].tolist())
y_test = np.array(df.loc[mask_test, 'logerror'].tolist())

data = (X_train, X_test, y_train, y_test)

In [ ]:
# save data
pickle.dump(data, open('../tmp/training_and_testing_data_2017.pkl', 'w'))

## train a XGboost regressor

In [ ]:
xgb = xgboost.XGBRegressor(n_estimators=200, 
                           #colsample_bytree=1, 
                           max_depth=3, 
                           #reg_alpha=0.001,
                           objective='reg:linear'
                          )

In [ ]:
xgb.fit(X_train, y_train)

In [ ]:
y_pred = xgb.predict(X_test)

In [ ]:
pd.Series(y_pred).hist()

In [ ]:
explained_variance_score(y_pred=y_pred, y_true=y_test)

In [ ]:
mean_absolute_error(y_pred=y_pred, y_true=y_test)

# #generate final model

In [ ]:
xgb.fit(np.concatenate((X_train, X_test), axis=0),np.concatenate((y_train, y_test), axis = 0))

# #Generate submission file

In [ ]:
# imputed missing values previously
y_pred = xgb.predict((df_pred[feats].values))

In [ ]:
# assume all 6 months have the same prediction results.
df_submission = df_pred[['parcelid','logerror']]

In [ ]:
df_submission['pred'] = y_pred

In [ ]:
# coalesce actual log error with prediction results

df_submission['201610'] = df_submission['logerror'].combine_first(df_submission['pred'])
df_submission['201611'] = df_submission['logerror'].combine_first(df_submission['pred'])
df_submission['201612'] = df_submission['logerror'].combine_first(df_submission['pred'])

df_submission['201710'] = df_submission['logerror'].combine_first(df_submission['pred'])
df_submission['201711'] = df_submission['logerror'].combine_first(df_submission['pred'])
df_submission['201712'] = df_submission['logerror'].combine_first(df_submission['pred'])

In [ ]:
del df_submission['logerror']
del df_submission['pred']

In [ ]:
df_submission.shape[0] == 2985217


In [ ]:
df_submission.to_csv('Users/fangjie/Downloads/submission_20171003_2017.csv', header=True, index=False)